# Interaction Model Examples

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fmrjerryjohns%2Fconnectedhomeip.git&urlpath=lab%2Ftree%2Fconnectedhomeip.git%2Fdocs%2Fguides%2Frepl%2FMatter+-+Basic+Interactions.ipynb&branch=python%2Fpersistent-storage">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This walks through the various interactions that can be initiated from the REPL towards a target using the Matter Interaction Model (IM) and Data Model (DM).

## Initialization

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

`ChipReplStartup.py` is run within the global namespace. This results in all of its imports being made available here.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [1]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

[1642616777472] [33129:1633205] CHIP: [DL] _Init
[1642616777472] [33129:1633354] CHIP: [DL] Platform main loop started.


──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

2022-01-19 10:26:18 johnsj-macbookpro1.roam.corp.google.com root[33129] CRITICAL Loading configuration from /tmp/repl-storage.json...


...

New FabricAdmin: FabricId: 1(1)


Fabric Admins have been loaded and are available at fabricAdmins

Creating default device controller on fabric 1...

Allocating new controller with FabricId: 1(1), NodeId: 1


Default CHIP Device Controller has been initialized to manage fabricAdmins[0], and is 
available as devCtrl

## Cluster Elements

The Interaction Model uses data model types that refer not just to the various base types defines in the spec, but types that correspond to structs/commands/events/attributes defined in each cluster specification. The cluster-specific types are referred to as 'cluster objects'. These are represented as Python dataclasses, with each field in the respective object equivalently named as a member within the dataclass.

### Namespaces

Objects in clusters are organized into namespaces. All clusters can be found under the `Clusters` namespace, with the appropriate cluster in upper camel case within that. (e.g `Clusters.TestCluster`).

Within that, `Commands`, `Structs` and `Attributes` delimit the respective types in the cluster.

_Example Struct:_

In [2]:
v = Clusters.TestCluster.Structs.SimpleStruct()
v.a = 20
v.b = True
v.c = Clusters.TestCluster.Enums.SimpleEnum.kValueA
v.d = b'1234'
v.e = 30
v.g = 23.234

v

SimpleStruct(
│   a=20,
│   b=True,
│   c=<SimpleEnum.kValueA: 1>,
│   d=b'1234',
│   e=30,
│   f=0,
│   g=23.234,
│   h=0.0
)

_Example Command:_

In [3]:
Clusters.TestCluster.Commands.TestAddArguments()

TestAddArguments(
│   arg1=0,
│   arg2=0
)

To get more information about the fields in these objects and their types, run:

In [4]:
matterhelp(Clusters.TestCluster.Commands.TestAddArguments)

╭──────── <class 'chip.clusters.Objects.TestCluster.Commands.TestAddArguments'> ─────────╮
│ def TestCluster.Commands.TestAddArguments(arg1: 'uint' = 0, arg2: 'uint' = 0) -> None: │
│                                                                                        │
│ TestAddArguments(arg1: 'uint' = 0, arg2: 'uint' = 0)                                   │
│                                                                                        │
│                  arg1 = 0                                                              │
│                  arg2 = 0                                                              │
│            cluster_id = 1295                                                           │
│            command_id = 4                                                              │
│            descriptor = ClusterObjectDescriptor(                                       │
│                             Fields=[                                                   │
│                                 ClusterObjectFieldDescriptor(                          │
│                                     Label='arg1',                                      │
│                                     Tag=0,                                             │
│                                     Type=<class 'chip.tlv.uint'>                       │
│                                 ),                                                     │
│                                 ClusterObjectFieldDescriptor(                          │
│                                     Label='arg2',                                      │
│                                     Tag=1,                                             │
│                                     Type=<class 'chip.tlv.uint'>                       │
│                                 )                                                      │
│                             ]                                                          │
│                         )                                                              │
│             is_client = True                                                           │
│ must_use_timed_invoke = False                                                          │
│              FromDict = def FromDict(data: dict):                                      │
│               FromTLV = def FromTLV(data: bytes):                                      │
│                 ToTLV = def ToTLV(self):                                               │
╰────────────────────────────────────────────────────────────────────────────────────────╯

### Nullable Fields

For fields that are nullable, they are represented as a `Typing.Union[Nullable, ...]`. This means that it can either be a `Nullable` type or the underlying type of the field.

When nullable, a field can either take on the value of the native type, or a value of `NullValue`.

In [5]:
a = Clusters.TestCluster.Structs.NullablesAndOptionalsStruct()
a.nullableInt = Clusters.Types.NullValue
a

NullablesAndOptionalsStruct(
│   nullableInt=Null,
│   optionalInt=None,
│   nullableOptionalInt=None,
│   nullableString=Null,
│   optionalString=None,
│   nullableOptionalString=None,
│   nullableStruct=Null,
│   optionalStruct=None,
│   nullableOptionalStruct=None,
│   nullableList=Null,
│   optionalList=None,
│   nullableOptionalList=None
)

### Optional Fields

If a field is optional, it is represented in the typing hints as a `Typing.Union[NoneType, ...]`. An optional field that isn't present has a value of `None`.

In [6]:
print(a.nullableOptionalInt)

None

### Defaults

Upon construction of a cluster object, the fields within that object are automatically initialized to type specific defaults as specified in the data model specification:

In [7]:
Clusters.TestCluster.Structs.SimpleStruct()

SimpleStruct(
│   a=0,
│   b=False,
│   c=0,
│   d=b'',
│   e='',
│   f=0,
│   g=0.0,
│   h=0.0
)

## IM Interactions

This section will walk through the various types of IM Interactions that are possible in the REPL.

### Commission and Setup Server

#### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [8]:
import time
import subprocess
subprocess.Popen(['pkill', '-f', 'chip-all-clusters-app'])
time.sleep(1)
process = subprocess.Popen('../../../out/debug/chip-all-clusters-app', stdout=subprocess.DEVNULL)
time.sleep(1)

#### Commission Target

Commission the target with a NodeId of 1.

In [9]:
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 2)

2022-01-19 10:26:20 johnsj-macbookpro1.roam.corp.google.com chip.CTL[33129] ERROR Unable to find country code, defaulting to WW
2022-01-19 10:26:20 johnsj-macbookpro1.roam.corp.google.com chip.SC[33129] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


ID: Allocated: 1
PersistNextKey 2
Established CASE with Device
ID: Allocated: 2
Node address has been updated
Commissioning complete


True

### Invoke Interaction

#### Basic Command (Success Response)

Let's send a basic command to turn on/off the light on Endpoint 1.

In [10]:
await devCtrl.SendCommand(2, 1, Clusters.OnOff.Commands.On())

The receipt of a successful status response will result in the command just returning successfully. Otherwise, an exception will be thrown.

#### Basic Command (Failure Response)

If we send the same command to an invalid endpoint, an exception is thrown. If an IM status code was received from the server, a `InteractionModelError` is thrown containing the IM status code:

In [11]:
try:
    await devCtrl.SendCommand(2, 100, Clusters.OnOff.Commands.On())
except Exception as e:
    pprint(e)

InteractionModelError(<Status.Failure: 1>)

#### Basic Command (Data Response)

Here's an example of a command that sends back a data response, and how that is presented:

In [12]:
await devCtrl.SendCommand(2, 1, Clusters.TestCluster.Commands.TestListInt8UReverseRequest([1, 3, 5, 7]))

TestListInt8UReverseResponse(
│   arg1=[
│   │   7,
│   │   5,
│   │   3,
│   │   1
│   ]
)

### Read Interaction

The `ReadAttribute` method on the `DeviceController` class can be used to read attributes from a target. The NodeId of the target is the first argument, followed by a list of paths that are expressed as cluster object namespaces to the respective slices of the data that is requested.

By default, the data is returned as a dictionary, with the top-level item representing the endpoint, then the cluster and the attribute. The latter two keys are expressed using cluster object namespaces.

#### Read 1 attribute:

In [13]:
a = await devCtrl.ReadAttribute(2, [Clusters.TestCluster.Attributes.Int16u])
a

{
│   1: {
│   │   <class 'chip.clusters.Objects.TestCluster'>: {
│   │   │   <class 'chip.clusters.Objects.TestCluster.Attributes.Int16u'>: 0
│   │   }
│   }
}

In [14]:
a[1][Clusters.TestCluster]

{
│   <class 'chip.clusters.Objects.TestCluster.Attributes.Int16u'>: 0
}

In [15]:
a[1][Clusters.TestCluster][Clusters.TestCluster.Attributes.Int16u]

0

#### Read 2 attributes:

In [16]:
await devCtrl.ReadAttribute(2, [Clusters.TestCluster.Attributes.Int16u, Clusters.TestCluster.Attributes.Boolean])

{
│   1: {
│   │   <class 'chip.clusters.Objects.TestCluster'>: {
│   │   │   <class 'chip.clusters.Objects.TestCluster.Attributes.Boolean'>: False,
│   │   │   <class 'chip.clusters.Objects.TestCluster.Attributes.Int16u'>: 0
│   │   }
│   }
}

#### Read the entirety of a cluster on an endpoint:

The path is represented as tuple of (endpoint, cluster)

In [17]:
await devCtrl.ReadAttribute(2, [(1, Clusters.OnOff)])

{
│   1: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   }
}

#### Read the entirety of a cluster across all endpoints:

In [18]:
await devCtrl.ReadAttribute(2, [Clusters.OnOff])

{
│   1: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   },
│   2: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: False,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   }
}

#### Read an endpoint:

In [19]:
await devCtrl.ReadAttribute(2, [2])

{
│   2: {
│   │   <class 'chip.clusters.Objects.Groups'>: {
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.NameSupport'>: 128,
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.ClusterRevision'>: 3
│   │   },
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: False,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   },
│   │   <class 'chip.clusters.Objects.Descriptor'>: {
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.DeviceList'>: [
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ServerList'>: [
│   │   │   │   4,
│   │   │   │   6,
│   │   │   │   29,
│   │   │   │   1030
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClientList'>: [],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.PartsList'>: [],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.OccupancySensing'>: {
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.Occupancy'>: 0,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.OccupancySensorType'>: 0,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.OccupancySensorTypeBitmap'>: 1,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.ClusterRevision'>: 2
│   │   }
│   }
}

#### Read the entire node:

In [20]:
#await devCtrl.ReadAttribute(2, [('*')])

#### Alternative 'Cluster' View

The above encapsulates each attribute as a 'cluster-object' key within the top-level cluster instance. Instead, an alternative view each attribute is represented as a field in the object can be retrieved by passing in `True` to the third argument:

In [21]:
await devCtrl.ReadAttribute(2, [2], True)

{
│   2: {
│   │   <class 'chip.clusters.Objects.Groups'>: Groups(
│   │   │   nameSupport=128,
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=3
│   │   ),
│   │   <class 'chip.clusters.Objects.OnOff'>: OnOff(
│   │   │   onOff=False,
│   │   │   globalSceneControl=True,
│   │   │   onTime=0,
│   │   │   offWaitTime=0,
│   │   │   startUpOnOff=0,
│   │   │   attributeList=None,
│   │   │   featureMap=0,
│   │   │   clusterRevision=4
│   │   ),
│   │   <class 'chip.clusters.Objects.Descriptor'>: Descriptor(
│   │   │   deviceList=[
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   serverList=[
│   │   │   │   4,
│   │   │   │   6,
│   │   │   │   29,
│   │   │   │   1030
│   │   │   ],
│   │   │   clientList=[],
│   │   │   partsList=[],
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=1
│   │   ),
│   │   <class 'chip.clusters.Objects.OccupancySensing'>: OccupancySensing(
│   │   │   occupancy=0,
│   │   │   occupancySensorType=0,
│   │   │   occupancySensorTypeBitmap=1,
│   │   │   pirOccupiedToUnoccupiedDelay=None,
│   │   │   pirUnoccupiedToOccupiedDelay=None,
│   │   │   pirUnoccupiedToOccupiedThreshold=None,
│   │   │   ultrasonicOccupiedToUnoccupiedDelay=None,
│   │   │   ultrasonicUnoccupiedToOccupiedDelay=None,
│   │   │   ultrasonicUnoccupiedToOccupiedThreshold=None,
│   │   │   physicalContactOccupiedToUnoccupiedDelay=None,
│   │   │   physicalContactUnoccupiedToOccupiedDelay=None,
│   │   │   physicalContactUnoccupiedToOccupiedThreshold=None,
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=2
│   │   )
│   }
}

#### Read Events:

A `ReadEvents` API exists that behaves similarly to the `ReadAttributes` API. It permits the same degrees of wildcard expression as its counterpart and follows the same format for expressing all wildcard permutations.

#### Read all events:

In [22]:
# Force an event to get emitted.
await devCtrl.SendCommand(2, 1, Clusters.TestCluster.Commands.TestEmitTestEventRequest())

await devCtrl.ReadEvent(2, [('*')])

[
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=0,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=71,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=4.0178029569121155e-41,
│   │   │   │   h=6.0841925560559e-310
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   )
]

### Subscription Interaction

To subscribe to a Node, the same `ReadAttributes` API is used to trigger a subscription, with a valid `reportInterval` tuple passed in being used as a way to indicate the request to create a subscription.

In [23]:
reportingTimingParams = (0, 2) # MinInterval = 0s, MaxInterval = 2s
subscription = await devCtrl.ReadAttribute(2, [(2, Clusters.OnOff)], True, reportingTimingParams)
subscription

<Subscription (Id=12846444511004550618)>

In [24]:
subscription.GetAttributes()

{
│   2: {
│   │   <class 'chip.clusters.Objects.OnOff'>: OnOff(
│   │   │   onOff=False,
│   │   │   globalSceneControl=True,
│   │   │   onTime=0,
│   │   │   offWaitTime=0,
│   │   │   startUpOnOff=0,
│   │   │   attributeList=None,
│   │   │   featureMap=0,
│   │   │   clusterRevision=4
│   │   )
│   }
}

#### Trigger Report

To trigger a report, let's alter the state of the on/off switch on EP1. That should trigger the generation of a set of attribute reports.

The `SubscriptionTransaction` object returned by `ReadAttribute` permits installing a callback that is invoked on any attribute report. A default callback is installed above that just dumps out the attribute data.

In [25]:
await devCtrl.SendCommand(2, 2, Clusters.OnOff.Commands.On())
time.sleep(1)

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>,
│   'Value': True
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>,
│   'Value': 0
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>,
│   'Value': True
}

In [26]:
await devCtrl.SendCommand(2, 2, Clusters.OnOff.Commands.Off())
time.sleep(1)

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>,
│   'Value': False
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>,
│   'Value': 0
}

In [27]:
subscription.Shutdown()

#### Subscribe to Events

In [28]:
reportingTimingParams = (0, 2) # MinInterval = 0s, MaxInterval = 2s
subscription = await devCtrl.ReadEvent(2, [()], reportingTimingParams)

In [29]:
subscription.GetEvents()

[
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=0,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=71,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=4.0178029569121155e-41,
│   │   │   │   h=6.0841925560559e-310
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   )
]

### Trigger Event

Force an event to get emitted, which after a short while, should generate a report and trigger the print out of the received event:

In [30]:
await devCtrl.SendCommand(2, 1, Clusters.TestCluster.Commands.TestEmitTestEventRequest())
time.sleep(3)

Received Event:


EventReadResult(
│   Header=EventHeader(
│   │   EndpointId=1,
│   │   Event=None,
│   │   EventNumber=1,
│   │   Priority=<EventPriority.DEBUG: 1>,
│   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   TimestampType=None
│   ),
│   Status=<Status.Success: 0>,
│   Data=TestEvent(
│   │   arg1=0,
│   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   arg3=False,
│   │   arg4=SimpleStruct(
│   │   │   a=71,
│   │   │   b=False,
│   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   d=b'',
│   │   │   e='',
│   │   │   f=0,
│   │   │   g=4.0178029569121155e-41,
│   │   │   h=6.0841925295455e-310
│   │   ),
│   │   arg5=[],
│   │   arg6=[]
│   )
)

In [31]:
subscription.Shutdown()

### Write Interaction

To write attribute data, the `WriteAttribute` API can be used. It requires a NodeId and a list of cluster object encapsulated data for the attribute being written.

In [32]:
await devCtrl.WriteAttribute(2, [ (1, Clusters.TestCluster.Attributes.Int16u(2)) ])

[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=1,
│   │   │   ClusterId=1295,
│   │   │   AttributeId=6
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [33]:
await devCtrl.ReadAttribute(2, [ (1, Clusters.TestCluster.Attributes.Int16u) ])

{
│   1: {
│   │   <class 'chip.clusters.Objects.TestCluster'>: {
│   │   │   <class 'chip.clusters.Objects.TestCluster.Attributes.Int16u'>: 2
│   │   }
│   }
}